In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd

In [2]:
sprint_data=pd.read_csv("/kaggle/input/springdataset/MOCK_DATA.csv")

In [3]:
print(sprint_data.head())
sprint_data.info()
sprint_data.describe()

   id   location  gender     contract complaints  tenure plandetails  \
0   1  Anchorage  Female    year-year        yes       1     premium   
1   2    Cordova  Female    year-year         No      13      custom   
2   3  Anchorage    Male  month-month         No      30     premium   
3   4     Haines    Male    year-year         No       2     premium   
4   5  Fairbanks    Male    year-year         No      28      custom   

   no_of_support_calls  monthlycharges  churn  
0                   36            2565      0  
1                   44            2890      0  
2                   23            3067      0  
3                    2            7419      0  
4                   32            5086      1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1000 non-null   int64 
 1   location             

,id,tenure,no_of_support_calls,monthlycharges,churn
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,15.466000,25.508000,5380.176000,0.504000
std,288.819436,8.695082,14.287129,2628.167539,0.500234
min,1.000000,1.000000,1.000000,1002.000000,0.000000
25%,250.750000,8.000000,14.000000,3025.500000,0.000000
50%,500.500000,15.000000,25.000000,5428.000000,1.000000
75%,750.250000,23.000000,38.000000,7689.000000,1.000000
max,1000.000000,30.000000,50.000000,9990.000000,1.000000


In [4]:
sprint_data.drop(["id"],axis=1,inplace = True)

In [5]:
encoded_sprint_data= pd.get_dummies (sprint_data[['location', 'gender', 'contract', 'complaints', 'plandetails']], drop_first=True)

In [6]:
encoded_sprint_data.head()

,location_Anchorage,location_Athens,location_Cordova,location_Fairbanks,location_Haines,gender_Male,contract_year-year,complaints_yes,plandetails_custom,plandetails_premium
0,True,False,False,False,False,False,True,True,False,True
1,False,False,True,False,False,False,True,False,True,False
2,True,False,False,False,False,True,False,False,False,True
3,False,False,False,False,True,True,True,False,False,True
4,False,False,False,True,False,True,True,False,True,False


In [7]:
bool_columns = ['location_Anchorage', 'location_Athens','location_Cordova','location_Fairbanks','location_Haines','gender_Male','contract_year-year','complaints_yes','plandetails_custom','plandetails_premium']

encoded_sprint_data[bool_columns] = encoded_sprint_data[bool_columns].astype(int)

In [8]:
encoded_sprint_data.head()

,location_Anchorage,location_Athens,location_Cordova,location_Fairbanks,location_Haines,gender_Male,contract_year-year,complaints_yes,plandetails_custom,plandetails_premium
0,1,0,0,0,0,0,1,1,0,1
1,0,0,1,0,0,0,1,0,1,0
2,1,0,0,0,0,1,0,0,0,1
3,0,0,0,0,1,1,1,0,0,1
4,0,0,0,1,0,1,1,0,1,0


In [9]:
int_data = sprint_data [['tenure', 'no_of_support_calls', 'monthlycharges', 'churn']]

In [10]:
sprintdata_final= pd.concat([int_data,encoded_sprint_data], axis=1)

In [11]:
sprintdata_final.head()

,tenure,no_of_support_calls,monthlycharges,churn,location_Anchorage,location_Athens,location_Cordova,location_Fairbanks,location_Haines,gender_Male,contract_year-year,complaints_yes,plandetails_custom,plandetails_premium
0,1,36,2565,0,1,0,0,0,0,0,1,1,0,1
1,13,44,2890,0,0,0,1,0,0,0,1,0,1,0
2,30,23,3067,0,1,0,0,0,0,1,0,0,0,1
3,2,2,7419,0,0,0,0,0,1,1,1,0,0,1
4,28,32,5086,1,0,0,0,1,0,1,1,0,1,0


In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score

 
x =sprintdata_final.drop('churn', axis=1)
y =sprintdata_final['churn']

#split data into train and test tests
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

model=xgb.XGBClassifier()
model.fit(x_train,y_train)

ypred= model.predict(x_test)

#Evaluate the model's performance using metrics like accuracy, precision, recall, and ROC-AUC.
accuracy = accuracy_score(y_test,ypred)
recall = recall_score(y_test,ypred)
f1 = f1_score(y_test,ypred)
precision = precision_score(y_test,ypred)


In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score
    
print('confusion matrix :',confusion_matrix(y_test,ypred))
print('classification report :',classification_report(y_test,ypred))
print('accuracy :',round(accuracy_score(y_test,ypred),2))
print('recall :',round(recall_score(y_test,ypred),2))
print('f1 :',round(f1_score(y_test,ypred),2))
print('precision:',round(precision_score(y_test,ypred),2))
print()

confusion matrix : [[69 91]
 [64 76]]
classification report :               precision    recall  f1-score   support

           0       0.52      0.43      0.47       160
           1       0.46      0.54      0.50       140

    accuracy                           0.48       300
   macro avg       0.49      0.49      0.48       300
weighted avg       0.49      0.48      0.48       300

accuracy : 0.48
recall : 0.54
f1 : 0.5
precision: 0.46

